In [3]:
%pip install geopandas rasterio matplotlib scikit-learn pandas pyimpute xgboost lightgbm

Note: you may need to restart the kernel to use updated packages.


# Species Distribution Data Loading and Preprocessing

### Load Species Distribution Data

Please note if this cell is re-run, it will take a very long time to process again!

Preprocess the Data in chunks (to avoid running out of RAM)

In [42]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import os
import tempfile

# File is removed for github upload; download .zip from Box or Drive and rename to below with appropriate folder hierarchy
# TODO: Save trimmed version of CSV and use that in github
df = pd.read_csv("data/GBIF_CardellinaPusilla.csv", sep='\t', lineterminator='\n')



C:\Users\marche\AppData\Local\Temp\ipykernel_17876\3293758763.py:9: DtypeWarning: Columns (10,14,19,23,24,25,39,40,41,43,45,46,48) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("data/GBIF_CardellinaPusilla.csv", sep='\t', lineterminator='\n')


In [31]:
# Load the GeoDataFrame from the GeoPackage file
import geopandas as gpd

pa = gpd.GeoDataFrame(df, 
    geometry = gpd.points_from_xy(df['decimalLatitude'], df['decimalLongitude']), 
    crs = 'EPSG:4326')

# Inspect the first few rows
print(gdf.head())


                                        occurrenceID      basisOfRecord  \
0  https://www.inaturalist.org/observations/13947...  HUMAN_OBSERVATION   
1  https://www.inaturalist.org/observations/10616...  HUMAN_OBSERVATION   
2  https://www.inaturalist.org/observations/11843...  HUMAN_OBSERVATION   
3  https://www.inaturalist.org/observations/34513725  HUMAN_OBSERVATION   
4  https://www.inaturalist.org/observations/12732366  HUMAN_OBSERVATION   

          eventDate   kingdom                       scientificName taxonRank  \
0  2022-09-28T12:24  Animalia  Cardellina pusilla (A.Wilson, 1811)   SPECIES   
1  2021-09-30T12:45  Animalia  Cardellina pusilla (A.Wilson, 1811)   SPECIES   
2  2022-05-23T20:54  Animalia  Cardellina pusilla (A.Wilson, 1811)   SPECIES   
3  2019-09-21T10:58  Animalia  Cardellina pusilla (A.Wilson, 1811)   SPECIES   
4  2018-05-20T08:13  Animalia  Cardellina pusilla (A.Wilson, 1811)   SPECIES   

   decimalLatitude  decimalLongitude countryCode individualCount  \


### Removing duplicates and NaN

We now check that there are no duplicate or `NaN` coordinates, as well as inspect the shapefile's attributes.

In [32]:
print("number of duplicates: ", pa.duplicated(subset='geometry', keep='first').sum())
print("number of NA's: ", pa['geometry'].isna().sum())
print("Coordinate reference system is: {}".format(pa.crs))
print("{} observations with {} columns".format(*pa.shape))

number of duplicates:  1352941
number of NA's:  0
Coordinate reference system is: EPSG:4326
1620779 observations with 11 columns


In [33]:
# Remove duplicates based solely on the 'geometry' column
species_distribution_unique = pa.drop_duplicates(subset=['geometry'])

# Reset index after removing duplicates
species_distribution_unique.reset_index(drop=True, inplace=True)


In [34]:
print("number of duplicates: ", species_distribution_unique.duplicated(subset='geometry', keep='first').sum())
print("number of NA's: ", species_distribution_unique['geometry'].isna().sum())
print("Coordinate reference system is: {}".format(species_distribution_unique.crs))
print("{} observations with {} columns".format(*species_distribution_unique.shape))


number of duplicates:  0
number of NA's:  0
Coordinate reference system is: EPSG:4326
267838 observations with 11 columns


#### Export the trimmed shapefile

In [40]:
# species_distribution_unique.to_csv("outputs/trimmed_CardellinaPusilla.csv", sep='\t')
species_distribution_unique.to_file('outputs/CardellinaPusilla.shp')  

#Climate Data Loading and Preprocessing

Load Climate Data

In [ ]:
import rasterio

# Example for loading a single climate variable
climate_variable_path = '/content/drive/My Drive/path_to_climate_variable.tif'
with rasterio.open(climate_variable_path) as src:
    climate_data = src.read(1)  # Reads the first band


Model Training and Assessment

In [ ]:
from pyimpute import load_training_vector, load_targets

# Assuming 'raster_features' is a list of paths to your climate raster files
train_xs, train_y = load_training_vector(species_distribution, raster_features, response_field='CLASS')


Train ML Classifiers

In [ ]:
# import machine learning classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

CLASS_MAP = {
    'rf': (RandomForestClassifier()),
    'et': (ExtraTreesClassifier()),
    'xgb': (XGBClassifier()),
    'lgbm': (LGBMClassifier())
    }
from pyimpute import impute
from sklearn import model_selection
# model fitting and spatial range prediction
for name, (model) in CLASS_MAP.items():
    # cross validation for accuracy scores (displayed as a percentage)
    k = 5 # k-fold
    kf = model_selection.KFold(n_splits=k)
    accuracy_scores = model_selection.cross_val_score(model, train_xs, train_y, cv=kf, scoring='accuracy')
    print(name + " %d-fold Cross Validation Accuracy: %0.2f (+/- %0.2f)"
          % (k, accuracy_scores.mean() * 100, accuracy_scores.std() * 200))

    # spatial prediction
    model.fit(train_xs, train_y)
    os.mkdir('outputs/' + name + '-images')
    impute(target_xs, model, raster_info, outdir='outputs/' + name + '-images',
           class_prob=True, certainty=True)

Map Species–Environment Relationship

In [ ]:
from pyimpute import impute

# Prepare target raster grids for prediction
target_xs, raster_info = load_targets(raster_features)
impute(target_xs, model, raster_info, outdir='/content/drive/My Drive/outputs', class_prob=True)


Visualize Model Predictions

In [ ]:
import matplotlib.pyplot as plt

# Example for visualizing one of the output probability maps
with rasterio.open('/content/drive/My Drive/outputs/probability_1.0.tif') as src:
    probability_map = src.read(1)

plt.imshow(probability_map, cmap='viridis')
plt.colorbar()
plt.title('Predicted Species Distribution')
plt.show()
